>># **Ciencia de Datos en Python**
### Proyecto Final
###### *Marco Tulio Escobar Muñoz*

In [ ]:
import numpy as np
import panda as pd
import matplotlib as mt
import seaborn as sns
import scikit-learn as sl

class my_regresion(self, x, y):
    def __init__():
        self.beta0 = []
        self.beta1 = []
        self.x = []
        self.y = []
        self.archivo = np.load("proyecto_training_data.npy")
        self.dsEntrenamiento = []
        self.dsEntrenamientoNan = []
        self.dsValidacion = []
        self.flMedia = float
        self.flValorMaximo = float
        self.flValorMinimo = float
        self.flRango = float
        self.flDesviacionEstandar = float
    
    def GetDataSet(self):
        long = len(self.archivo)
        long81 = (long * 0.80) + 1
        long80 = long * 0.80
        long20 = long * 0.20
        self.dsEntrenamiento = archivo[0:long81]
        self.dsValidacion = archivo[long80:]
        
        return self.dsEntrenamiento, self.dsValidacion
    
    def GetExploraionData(self):
        self.dsEntrenamientoNan = self.dsEntrenamiento
        self.dsEntrenamientoNan[np.isnan(self.dsEntrenamientoNan)] = 0
        self.flMedia = np.nanmean(self.dsEntrenamiento)
        self.flValorMaximo = np.nanmax(self.dsEntrenamiento)
        self.flValorMinimo = np.nanmin(self.dsEntrenamiento)
        self.flRango = np.ptp(self.dsEntrenamientoNan)
        self.flDesviacionEstandar = np.nanstd(self.dsEntrenamiento)
        
        return self.flMedia, self.flValorMaximo, self.flValorMinimo, self.flRango, self.DesviacionEstandar